# Tutorial, Part I: Candidate Extraction

In this example, we'll be writing an application to extract **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  At core, we will be constructing a model to classify _candidate chemical-disease (C-D) relation mentions_ as either true or false.  To do this, we first need a set of such candidates- in this notebook, we'll use `DDLite` utilities to extract these candidates.

## Loading the Corpus

First, we will load and pre-process the corpus, storing it for convenience in a `Corpus` object

### Configuring a document parser

We'll start by defining a `DocParser` class to read in Pubmed abstracts from [Pubtator]([Pubtator](http://www.ncbi.nlm.nih.gov/CBBresearch/Lu/Demo/PubTator/index.cgi)), where they are stored along with "gold" (i.e. hand-annotated by experts) *chemical* and *disease mention* annotations. We'll use the `XMLDocParser` class, which allows us to use [XPath queries](https://en.wikipedia.org/wiki/XPath) to specify the relevant sections of the XML format.

_Note that we are newline-concatenating text from the title and abstract together for simplicity, but if we wanted to, we could easily extend the `DocParser` classes to preserve information about document structure._

In [1]:
from ddlite_parser import XMLDocParser
xml_parser = XMLDocParser(
    path='data/CDR_DevelopmentSet.xml',
    doc='.//document',
    text='.//passage/text/text()',
    id='.//id/text()',
    keep_xml_tree=True)

### Selecting a sentence parser

Next, we'll use an NLP preprocessing tool to split the `Document` objects into sentences, tokens, and provide annotations--part-of-speech tags, dependency parse structure, lemmatized word forms, etc.--for these sentences.  Here we use the default `SentenceParser` class.

In [2]:
from ddlite_parser import SentenceParser
sent_parser = SentenceParser()

### Pre-processing & loading the corpus

Finally, we'll put this all together using a `Corpus` object, which will execute the parsers and store the results as an iterator:

In [3]:
from ddlite_parser import Corpus
corpus = Corpus(xml_parser, sent_parser)

Parsing documents...
Parsing sentences...


In [4]:
for doc, sentences in corpus:
    print doc
    print "\n"
    print sentences[0]
    break

Document(id='6794356', file='CDR_DevelopmentSet.xml', text='Tricuspid valve regurgitation and lithium carbonate toxicity in a newborn infant.\nA newborn with massive tricuspid regurgitation, atrial flutter, congestive heart failure, and a high serum lithium level is described. This is the first patient to initially manifest tricuspid regurgitation and atrial flutter, and the 11th described patient with cardiac disease among infants exposed to lithium compounds in the first trimester of pregnancy. Sixty-three percent of these infants had tricuspid valve involvement. Lithium carbonate may be a factor in the increasing incidence of congenital heart disease when taken during early pregnancy. It also causes neurologic depression, cyanosis, and cardiac arrhythmia when consumed prior to delivery.', attribs={'root': <Element document at 0x108093a50>})


Sentence(id='6794356-0', words=[u'Tricuspid', u'valve', u'regurgitation', u'and', u'lithium', u'carbonate', u'toxicity', u'in', u'a', u'newbor

## Writing a basic candidate extractor

Next, we'll write a basic function to extract **candidate disease mentions** from the corpus.  For this first attempt, we'll just write a function that checks for matches against a list (or _"dictionary"_) of disease phrases, constructed using some pre-compiled ontologies ([UMLS](https://www.nlm.nih.gov/research/umls/), [ORDO](http://www.orphadata.org/cgi-bin/inc/ordo_orphanet.inc.php), [DOID](http://www.obofoundry.org/ontology/doid.html), [NCBI Diseases](http://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/); see `tutorial/data/diseases.py`).

We'll do this using a `CandidateSpace` object--which defines the basic candidates we consider, in this case n-grams up to a certain length--and a `Matcher` object, which filters this candidate space down.

In [5]:
from load_dictionaries import load_disease_dictionary
from ddlite_candidates import Ngrams
from ddlite_matchers import DictionaryMatch

# Load the disease phrase dictionary
diseases = load_disease_dictionary()
print "Loaded %s disease phrases!" % len(diseases)

# Define a candidate space
ngrams = Ngrams(n_max=3)

# Define a matcher
matcher = DictionaryMatch(d=diseases, longest_match_only=False)

Loaded 507899 disease phrases!


Note that we set `longest_match_only=False`, which means that we _will_ consider subsequences of phrases that match our dictionary.

The `Ngrams` operator is applied over our `Sentence` objects and returns `Ngram` objects, and the `Matcher` then filters these, so we apply our operators over the corpus (also storing the results in our `Corpus` object):

In [6]:
corpus.extract_candidates(ngrams, matcher)
candidates = list(corpus.iter_candidates())
candidates[:5]

[<Ngram("blood pressure", id=1969772-5:640-653, chars=[640,653], words=[2,3]),
 <Ngram("pressure", id=1969772-5:646-653, chars=[646,653], words=[3,3]),
 <Ngram("anaesthesia", id=1969772-5:925-935, chars=[925,935], words=[44,44]),
 <Ngram("hypotension", id=1969772-4:562-572, chars=[562,572], words=[9,9]),
 <Ngram("Mean arterial pressure", id=1969772-6:938-959, chars=[938,959], words=[0,2])]

## Evaluating our candidate recall on gold annotations

Next, we'll test our _candidate recall_--in other words, how many of the true disease mentions we picked up in our candidate set--using the gold annotations in our dataset.

The XML documents that we loaded using the `XMLDocParser` also contained annotations (this is why we kept the full xml tree using `keep_xml_tree=True`).  We'll load these annotations and map them to `Ngram` objects over our parsed sentences, that way we can easily compare our extracted candidate set with the gold annotations.  The code is fairly simple (see `tutorial/util.py`); note that we filter to only keep _disease_ annotations, and that the candidates should be uniquely identified by their `id` attribute:

In [12]:
from utils import collect_pubtator_annotations
gold = []
for doc, sents in corpus:
    gold += [a for a in collect_pubtator_annotations(doc, sents) if a.metadata['type'] == 'Disease']
gold = frozenset(gold)

Now, we can get candidate recall using simple set operations (note that the candidates are identified based on their `id` attribute):

In [13]:
def print_candidate_stats(candidates, gold):
    nc = len(candidates)
    ng = len(gold)
    print "# of candidates\t\t= %s" % nc
    print "Candidate recall\t= %0.3f" % (len(gold.intersection(candidates)) / float(ng),)
    print "Candidate precision\t= %0.3f" % (len(gold.intersection(candidates)) / float(nc),)

print_candidate_stats(candidates, gold)

# of candidates		= 6944
Candidate recall	= 0.716
Candidate precision	= 0.438


We note that our focus in this stage is on **acheiving high candidate recall, without considering an impractically large candidate set**.  Our main focus after this stage will be on training a classifier to select which candidates are true; this will raise precision while hopefully keeping recall high.  _Note however that candidate recall is an upper bound for the recall of this classifier!_

So, we have some work to do.

## Using the `Viewer` to inspect data

Next, we'll take a random set of all the sentences where there are gold annotations not in our candidate set, and use the `Viewer` class--here, specifically, the `SentenceNgramViewer`--to inspect the data

In [22]:
from random import sample

# Collect indexed by sentence id
candidates_by_sid = defaultdict(list)
gold_by_sid       = defaultdict(list)
sentences_by_sid  = {}
for c in corpus.iter_candidates():
    candidates_by_sid[c.sent_id].append(c)
for g in gold:
    gold_by_sid[g.sent_id].append(g)
for s in corpus.iter_sentences():
    sentences_by_sid[s.id] = s
    
# Take random sample
NS_MAX = 100
sids   = gold_by_sid.keys()
shuffle(sids)
sents_sample = []
cands_sample = []
gold_sample  = []  
for sid in sids:
    if len(candidates_by_sid[sid]) < len(gold_by_sid[sid]):
        sents_sample.append(sentences_by_sid[sid])
        cands_sample += candidates_by_sid[sid]
        gold_sample  += gold_by_sid[sid]
        if len(sents_sample) == NS_MAX:
            break

## TO-DO:

### Clean the above up by extending `Corpus` object!

### Split into a `Candidates` object?

In [28]:
from ddlite_viewer import SentenceNgramViewer
sv        = SentenceNgramViewer(sents_sample, cands_sample, lambda c : c.sent_id, gold=gold_sample, n_max=100)
sv.render(n_per_page=3, height=225)

Viewer 
 
 
 
 
 
 Earlier studies have demonstrated that antagonism of sigma1 receptors attenuates the convulsive , lethal, locomotor stimulatory and rewarding actions of cocaine in mice. 
 
 The latter was characterized by a pronounced increase in protein excretion , especially proteins with molecular weight greater than 40,000 Da. 
 
 Anginal episodes were rare: only one patient had angina (during 5-FU infusion). 
 
 
 
 
 
 However, fluid attenuated inversion recovery (FLAIR) sequences frequently revealed persistent white matter abnormalities . 
 
 The patients were interviewed with emphasis on neuropathic symptoms . 
 
 Cardiotoxic and possible leukemogenic effects of adriamycin in nonhuman primates. 
 
 
 
 
 
 Clinical use of the anthracycline doxorubicin (DOX) is limited by its cardiotoxic effects, which are attributed to the induction of apoptosis. 
 
 A series of agents acting primarily on neurotransmitters other than dopamine were then tested in combination with L-DOPA to see if the dyskinetic movements would be modified. 
 
 RATIONALE AND OBJECTIVES: The light-enhanced startle paradigm (LES) is suggested to model anxiety , because of the non-specific cue and the long-term effect. 
 
 
 
 
 
 Force overflow and levodopa-induced dyskinesias in Parkinson's disease . 
 
 There is no difference in breast-cancer frequency between the placebo (22 cases) and tamoxifen (19) arms. 
 
 Prostaglandin E2 showed a hyperalgesic effect at doses of 1 pg to 10 ng/mouse, but the effect lasted shorter (3-30 min) than that of prostaglandin D2. 
 
 
 
 
 
 Nephrotoxic effects of aminoglycoside treatment on renal protein reabsorption and accumulation. 
 
 This article summarizes all the cases published, creating a comprehensive review of the current knowledge and approach to SIAT . 
 
 These results show that reserpine produces different and abnormal movements , which are related to dose and schedule employed and can be considered as parkinsonian -like and tardive dsykinesia signs. 
 
 
 
 
 
 Previous studies suggested the possiblility of increased propensity for thromboembolic episodes in patients possessing the A antigen. 
 
 We investigated changes in urinary NGF and PGs in women with OAB . 
 
 After the end of the treatment, the number and the size of induced PAS positive preneoplastic foci was significantly reduced when PB was given simultaneously with DEN for 4 and 6 weeks. 
 
 
 
 
 
 Delayed leukoencephalopathy with stroke -like presentation in chemotherapy recipients. 
 
 However, the replacement of last two doses of mesna with saline or all of the mesna doses with dexamethasone did not prevent HC . 
 
 It is likely that the hypersensitivity ( eosinophilic ) myocarditis was related to dobutamine infusion therapy. 
 
 
 
 
 
 These results fail to support the notion that levodopa-induced dyskinesias are caused by overstimulation of a separate group of dopamine receptors. 
 
 Excitotoxicity leading to neuronal cell death in the affected areas is a major consequence of seizures at the cellular level. 
 
 In Parkinson's disease + LID , the force involved in pressing down the object before lifting was significantly increased by levodopa (by 61%, P < 0.05). 
 
 
 
 
 
 CONCLUSION: This study showed that spironolactone can reduce potassium requirements and prevent hypokalemia by reducing urinary potassium loss in neutropenic patients on AmB treatment. 
 
 Whether iso-osmolar contrast medium is less nephrotoxic than low-osmolar contrast medium in high-risk patients is uncertain. 
 
 The objective of this study was to evaluate the direct health-care costs and management of RIHA during treatment of CHC in a clinical trial setting. 
 
 
 
 
 
 Fewer than 50 cases of SIAT have been described in the literature. 
 
 ATPase staining of the diaphragm, scalenus medius, and gastrocnemius showed type IIb fiber atrophy in the steroid groups and also diaphragmatic type IIa atrophy with T, whereas histologic examinations revealed a normal muscular p

## Composing a better candidate extractor

In [16]:
from ddlite_matchers import Union
from load_dictionaries import load_acronym_dictionary

# Load the disease phrase dictionary
acronyms = load_acronym_dictionary()
print "Loaded %s acronyms!" % len(acronyms)

# Define a new matcher
matcher = Union(
    DictionaryMatch(d=diseases, longest_match_only=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
corpus.extract_candidates(ngrams, matcher)
print_candidate_stats(list(corpus.iter_candidates()), gold)

Loaded 36904 acronyms!
# of candidates		= 7530
Candidate recall	= 0.747
Candidate precision	= 0.421


Next, we try using the `Concat` and `RegexMatch` operators:

In [21]:
from ddlite_matchers import Concat, RegexMatchEach
matcher = Union(
    Concat(
        RegexMatchEach(rgx=r'JJ*', attrib='poses'),
        DictionaryMatch(d=diseases, longest_match_only=False),
    left_required=False),
    DictionaryMatch(d=acronyms, ignore_case=False))

# Extract a new set of candidates
corpus.extract_candidates(ngrams, matcher)
print_candidate_stats(list(corpus.iter_candidates()), gold)

# of candidates		= 9306
Candidate recall	= 0.789
Candidate precision	= 0.360


### More coming here...

We've increased the candidate recall (on the development set) by 7.3% using some simple compositional `Matcher` operators.  We'll be adding more here soon!